# Работа с .csv файлами

Ранее пользовались хранилищем в виде простых форматов .txt и .dat. Для работы с ними приходилось либо писать функции обработки или примитивно встроенными в numpy сохранением и загрузкой. Теперь рассмотрим более удобный как для пользователя, так и для программиста способом хранения - csv.<br>
По своей сути он близок к формату данных, который мы создавали раннее - сверху через разделитель записываются ключи, ниже построчно через разделитель выводятся данные. Его удобство во вшитом формате, что позволяет легко использовать его со сторонними программами, например, Microsoft Excel<br>
Подобно numpy, всего один метод позволяет создать и заполнить файл, и ещё один прочитать содержимое

In [ ]:
import pandas as pd

file_name = 'file_name.csv'

In [ ]:
frame = pd.DataFrame({'a': [1, 2],
                      'b': [3, 4]})
frame.to_csv(file_name, mode='w', sep=',', index=True, header=True, encoding='utf-8', errors='ignore', decimal='.')

Файл можно посмотреть в папке с файлом программы. Первым аргументом указан путь к файлу (абсолютный или относительный), режим задаёт набор разрешённых операций, среди которых:<br>
- ‘w’, создание или перезапись существующего файла<br>
- ‘x’, создание файла, возвращает ошибку при существовании<br>
- ‘a’, дописывает в существующий или создаёт новый файл<br>

<i>sep</i> задаёт разделитель между числами (по умолчанию - запятая)<br>
<i>'index'</i> и <i>'header'</i> отвечают за создание столбца индексов и строки заголовков. Полезно в следующей последовательности: <br>
1) Создаём пустой DataFrame с заголовками, соответствующим измеряемым величинам, с `header=True`, `index=False`
2) Сохраняем его в файл
3) При получении новых данных, формируем из них новый DF и записываем с `mode='a'`, `header=False`, `index=False`, то есть добавим только значения  <br>

<i>encoding</i> задаёт кодировку файла<br>
<i>errors</i> помогает избежать ошибок при чтении символов с неправильной кодировкой<br>
<i>decimal</i> изменяет разделитель целой и дробной части чисел

Посмотрим добавление новых данных:

In [ ]:
new_frame = pd.DataFrame({'a': [1.1],
                          'b': [3.1]})
new_frame.to_csv(file_name, mode='a', header=False, encoding='utf-8', errors='ignore')

В файле появилась новая строка с данными, но с неправильным индексом. Самый простой способ избежать этого - не писать индексы вообще. Или вручную отслеживать размер хранилища и менять индексы в новых DataFrame

In [ ]:
index = 4
new_frame = pd.DataFrame({'a': [1.1],
                          'b': [3.1]},
                         index=[index])
index += 1
new_frame

Схожим образом выполняется чтение данных из файла

In [ ]:
df = pd.read_csv(file_name, sep=',', usecols=['a', 'b'], 
                 skip_blank_lines=True, chunksize=None, 
                 decimal='.', encoding='utf-8', 
                 skiprows=range(0, 0))


<i>file_name, sep, decimal, encoding</i> совпадают с `df.to_csv`<br>
<i>usecols</i> ограничивает получаемые столбцы данных<br>

In [ ]:
df = pd.read_csv(file_name, usecols=['a'])
df

<i>skip_blank_lines</i> пропускает пустые строки<br>
<i>chunksize</i> позволяет читать файл частями, размер которых задаётся этим ключом. Это оптимизирует использование памяти компьютера, если не требуется загружать всю базу данных. Будет полезным при отображении на графике ранее обработанных и записанных в файл данных

In [ ]:
chunks = pd.read_csv(file_name, chunksize=1)
for df in chunks:
    print(df)

<i>skiprows</i> позволяет пропустить ранее прочитанные данные или неинтересные фрагменты хранилища. Принимает номер или контейнер с числами

In [ ]:
df = pd.read_csv(file_name, skiprows=range(2, 3))
df

Рассмотрим конкретные примеры:<br>
Пусть имеется набор параметров в виде словаря, за которыми мы следили. После сохранили весь DataFrame в файл(допустим большой). В какой-то момент нам потребовался определённый участок это базы, причём мы точно знаем где он находится - например, по времени начала и конца смогли вычислить номера строк. Из этого куска извлечём значения с постоянной(уже повторяющейся) температурой и запишем их в новый файл<br>
Начнём с генерации той самой базы

In [ ]:
from random import randint

data = {'T': [300, 300, 300, 250, 250, 220, 200, 200, 200, 150, 150, 150, 150, 100],
        'U': list(range(14)),
        'I': [randint(0, int(pow(i*10, 0.5))) for i in range(14)]}
data = pd.DataFrame(data)
data

Сохраним его в файл

In [ ]:
data.to_csv(file_name, header=True, index=False, mode='w')

Для экономии ресурсов будем читать файл с пропуском первых 3 и последних 5 строк

In [ ]:
new_data = pd.read_csv(file_name, skiprows=list(range(1, 4)) + list(range(10, 15)))
new_data

Теперь стоит задача разработать алгоритм отбора данных согласно обозначенному условию повторяющейся температуры. Предлагаю следующий вариант: собираем список уникальных значений температур и проверяем хранилище на количество повторений

In [ ]:
temps = new_data['T'].unique()
temps

Если значение встречается лишь 1 раз, то удаляем соответствующие ему строки методом `df.drop(...)`

In [ ]:
for T in temps:
    print(new_data['T'].loc[new_data['T'] == T])

Из трёх DataFrame нужно выбрать только те, число строк в которых не превышает 1. В этом поможет метод `df.count()`

In [ ]:
for T in temps:
    if new_data['T'].loc[new_data['T'] == T].count() == 1:
        print(T)

Мы убедились, что верно находим температуру для удаления. Для метода `df.drop(...)` понадобятся индексы строк. Их сможем получить, запросив методом `df.index` найденные строки

In [ ]:
for T in temps:
    if new_data['T'].loc[new_data['T'] == T].count() == 1:
        print(new_data.loc[new_data['T'] == T].index)

Причём, индексы строк не зависят от рассматриваемого DataFrame (<i>new_data, new_data['T']</i>). Осталось добавить последний шаг

In [ ]:
for T in temps:
    if new_data['T'].loc[new_data['T'] == T].count() == 1:
        new_data.drop(new_data.loc[new_data['T'] == T].index, inplace=True)
        
new_data

Получили требуемый фрагмент. Сохраним его без индексов

In [ ]:
new_data.to_csv('filtered_data.csv', mode='w', index=False)

# Дополнительно

Были рассмотрены наиболее используемые методы и функции программной библиотеки `Pandas`, но есть и другие полезные представители, о которых сейчас пойдёт речь

In [ ]:
data = pd.DataFrame({'Pump_1': ['On', 'On', 'On', 'On', 'Off', 'Off', 'Off'],
                     'Pump_2': ['Off', 'Off', 'Off', 'Off', 'On', 'On', 'On'],
                     'Pressure': list(range(21))[::-3],
                     'Time': list(range(1, 8))})
data

`df.combine(other, func, fill_value)` Весьма полезная функция, позволяющая попарно применить любую функцию, передаваемую в <i>func</i>. Последним аргументом можно передать значение, которое займёт место недостающего элемента в контейнерах(схожа с методом `df.apply`, но расширяет возможности для использования с другим df)

In [7]:
data['Pressure'].combine(data['Time'], lambda x, y: x*y)

0    20
1    34
2    42
3    44
4    40
5    30
6    14
dtype: int64

При работе с большими словарями зачастую требуется выделять из них отдельные столбцы или строки с некоторыми параметрами. Для первого подойдёт перечисление ключей в виде списка:

In [ ]:
data[['Pump_1', 'Pump_2']]

Для второго метод `df.isin()`

In [ ]:
data[data['Pump_1'].isin(['On'])]

Он работает аналогично условиям, использованным раннее, но добавляет возможность искать сразу несколько элементов

In [ ]:
print(data['Pump_1'].isin(['On']), '\n')
print(data['Pump_1'] == 'On', '\n')
data['Pump_1'].isin(['On', 'Off'])

Если возникает необходимость получить элемент из DataFrame без запросов точных координат, например, через условие, то вывод будет перегруженным

In [ ]:
data.loc[data['Pressure'] < 5, ['Time']]

Убрать индекс, название столбца поможет метод `df.squeeze(type)`

In [ ]:
print(data.loc[data['Pressure'] < 5, ['Time']].squeeze(), '\n')
print(data.loc[data['Pressure'] < 5, ['Time']].squeeze('columns'), '\n')
print(data.loc[data['Pressure'] < 5, ['Time']].squeeze('rows'))

DataFrame можно транспонировать методом `T`

In [ ]:
data.T

`df.nlargest(n, column)`, `df.nsmallest(n, column)` возвращают несколько строк с самыми большими или малыми значениями в указанной колонке, причём отсортированными

In [ ]:
print(data.nlargest(2, 'Pressure'), '\n')
print(data.nsmallest(3, 'Pressure'), '\n')

`df.rename(columns=None, index=None, errors='ignore', inplace=False)` отвечает за переименование столбцов или индексов в DataFrame

In [ ]:
data.rename(columns={'Pump_1': 'Давление'}, inplace=True)
data

Больше функций и методов можно увидеть в официальной документации, а некоторые редко используемые расписанны на https://proglib.io/p/25-vozmozhnostey-pandas-o-kotoryh-vy-ne-znali-2022-02-28

## Excel

Вместо csv фалйов можно использовать excel. Для демонстрации воспользуемся ещё одной библиотекой `seaborn`

In [ ]:
import pandas as pd
import seaborn as sns

diamonds = sns.load_dataset("diamonds")[:100]
tips = sns.load_dataset("tips")[:100]

with pd.ExcelWriter("data.xlsx", mode='w', datetime_format='YYYY-MM-DD HH:MM:SS', date_format='YYYY-MM-DD') as writer:
    diamonds.to_excel(writer, sheet_name="diamonds")
    tips.to_excel(writer, sheet_name="tips")

with pd.ExcelWriter("data.xlsx", mode='a', if_sheet_exists='new') as writer:
    tips.to_excel(writer, sheet_name="tips")

Записываем базы данных в 2 переменных. При помощи класса `pd.ExcelWriter(...)` создаём или открываем файл<br>
Первый аргумент - путь с названием файлом<br>
<i>mode</i> - режим 'w' соответствует созданию нового файла, 'a' добавление в существующий<br>
<i>if_sheet_exists</i> только с <i>mode='a'</i>, действие при существующей странице с таким названием:<br>
- ‘error’ - поднятие ошибки ValueError<br>
- ‘new’ - создать новую страницу с изменённым названием<br>
- ‘replace’ - перезапись файла<br>
- ‘overlay’ - дописывание файлв<br>

<i>datetime_format, date_format</i> задают сохраняемый формат даты и времени

In [ ]:
from datetime import date, datetime  

with pd.ExcelWriter("data.xlsx", mode='w', datetime_format='YYYY-MM-DD HH:MM:SS', date_format='YYYY-MM-DD') as writer:
    df = pd.DataFrame([[date(2014, 1, 31), date(1999, 9, 24)],
                       [datetime(1998, 5, 26, 23, 33, 4), datetime(2014, 2, 28, 13, 5, 13)],],
                      index=["Date", "Datetime"],
                      columns=["X", "Y"])
    df.to_excel(writer, sheet_name='1')